In [14]:
import pandas as pd
import numpy as np
import lightgbm
import catboost
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler,StandardScaler,OrdinalEncoder,OneHotEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

In [15]:
train1 = pd.read_csv("datasets/median_data_train.csv")
test1 = pd.read_csv("datasets/median_data_test.csv")

In [16]:
train1.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)
test1.drop(["Unnamed: 0","tarih"],axis=1,inplace=True)

In [17]:
test1.head()

,ilce,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,izmir-aliaga,0,38.7996,26.9707,14.15,24.85,0.0,83.10,167.85,4.25,1.0,14.50
1,izmir-aliaga,0,38.7996,26.9707,15.20,9.05,0.0,79.85,159.10,1.65,1.0,15.30
2,izmir-aliaga,0,38.7996,26.9707,14.85,52.05,0.0,69.75,133.75,4.00,1.0,14.50
3,izmir-aliaga,0,38.7996,26.9707,14.95,81.60,0.0,78.45,175.60,8.05,1.0,14.35
4,izmir-aliaga,0,38.7996,26.9707,14.85,52.35,0.0,68.45,187.80,6.90,1.0,13.60


In [18]:
train1.head()

,ilce,bildirimsiz_sum,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,t_apparent:C
0,izmir-aliaga,5.0,0.0,38.7996,26.9707,12.85,50.75,0.0,88.00,170.00,4.75,1.0,13.00
1,izmir-aliaga,3.0,0.0,38.7996,26.9707,12.45,36.40,0.0,83.90,106.85,3.00,1.0,12.60
2,izmir-aliaga,2.0,0.0,38.7996,26.9707,12.10,57.30,0.0,78.90,120.80,3.45,1.0,12.00
3,izmir-aliaga,0.0,0.0,38.7996,26.9707,14.15,48.05,0.0,67.85,113.95,4.55,1.0,14.05
4,izmir-aliaga,8.0,0.0,38.7996,26.9707,14.20,22.65,0.0,79.20,178.85,3.95,1.0,14.20


In [19]:
ord_coder = OrdinalEncoder()
ohe = OneHotEncoder()
sc = StandardScaler()

In [20]:
X = train1.drop("bildirimsiz_sum",axis=1)
Y = train1["bildirimsiz_sum"]

In [21]:
dummies = ohe.fit_transform(X[["ilce"]])
dummies = dummies.toarray()
features = ohe.get_feature_names_out(["ilce"])
dummy = pd.DataFrame(data=dummies,columns=features)

In [22]:
X = X.drop("ilce",axis=1)

In [23]:
X = pd.concat([X,dummy],axis=1)

In [24]:
X

,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,...,ilce_manisa-koprubasi,ilce_manisa-kula,ilce_manisa-salihli,ilce_manisa-sarigol,ilce_manisa-saruhanli,ilce_manisa-sehzadeler,ilce_manisa-selendi,ilce_manisa-soma,ilce_manisa-turgutlu,ilce_manisa-yunusemre
0,0.0,38.7996,26.9707,12.85,50.75,0.0,88.00,170.00,4.75,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,38.7996,26.9707,12.45,36.40,0.0,83.90,106.85,3.00,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,38.7996,26.9707,12.10,57.30,0.0,78.90,120.80,3.45,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,38.7996,26.9707,14.15,48.05,0.0,67.85,113.95,4.55,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,38.7996,26.9707,14.20,22.65,0.0,79.20,178.85,3.95,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51601,0.0,39.1881,27.6100,9.75,70.15,0.0,86.80,225.30,1.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51602,1.0,39.1881,27.6100,11.75,24.60,0.0,74.65,153.05,2.50,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51603,0.0,39.1881,27.6100,12.50,70.05,0.0,71.65,152.70,3.80,1.00,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51604,1.0,39.1881,27.6100,14.00,66.35,0.0,76.75,155.70,2.90,1.85,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [47]:
# X["ilce"] = ord_coder.fit_transform(X[["ilce"]])

In [25]:
minmax = MinMaxScaler()

In [30]:
cols = dummy.columns.to_list()

In [32]:
cols.append("bildirimli_sum")

In [35]:
mm_columns = X.drop(cols,axis=1).columns

In [36]:
X[mm_columns] = minmax.fit_transform(X[mm_columns])

In [37]:
X

,bildirimli_sum,lat,lon,t_2m:C,effective_cloud_cover:p,global_rad:W,relative_humidity_2m:p,wind_dir_10m:d,wind_speed_10m:ms,prob_precip_1h:p,...,ilce_manisa-koprubasi,ilce_manisa-kula,ilce_manisa-salihli,ilce_manisa-sarigol,ilce_manisa-saruhanli,ilce_manisa-sehzadeler,ilce_manisa-selendi,ilce_manisa-soma,ilce_manisa-turgutlu,ilce_manisa-yunusemre
0,0.0,0.68601,0.260122,0.484581,0.5075,0.0,0.858989,0.473528,0.309028,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.68601,0.260122,0.475771,0.3640,0.0,0.810811,0.295214,0.187500,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.68601,0.260122,0.468062,0.5730,0.0,0.752056,0.334604,0.218750,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.68601,0.260122,0.513216,0.4805,0.0,0.622209,0.315262,0.295139,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.68601,0.260122,0.514317,0.2265,0.0,0.755582,0.498518,0.253472,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51601,0.0,1.00000,0.509255,0.416300,0.7015,0.0,0.844888,0.629677,0.083333,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51602,1.0,1.00000,0.509255,0.460352,0.2460,0.0,0.702115,0.425667,0.152778,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51603,0.0,1.00000,0.509255,0.476872,0.7005,0.0,0.666863,0.424679,0.243056,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
51604,1.0,1.00000,0.509255,0.509912,0.6635,0.0,0.726792,0.433150,0.180556,0.010235,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [38]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.30, random_state=42)

In [39]:
classifier = catboost.CatBoostRegressor(learning_rate=0.0105,iterations=1000,task_type="GPU")

In [40]:
classifier.fit(X_train,Y_train)

0:	learn: 5.4494241	total: 55.9ms	remaining: 55.8s
1:	learn: 5.4421036	total: 81.6ms	remaining: 40.7s
2:	learn: 5.4353748	total: 110ms	remaining: 36.6s
3:	learn: 5.4289738	total: 145ms	remaining: 36.1s
4:	learn: 5.4224253	total: 168ms	remaining: 33.5s
5:	learn: 5.4158746	total: 196ms	remaining: 32.4s
6:	learn: 5.4098312	total: 238ms	remaining: 33.8s
7:	learn: 5.4036889	total: 263ms	remaining: 32.6s
8:	learn: 5.3977145	total: 321ms	remaining: 35.4s
9:	learn: 5.3919348	total: 416ms	remaining: 41.2s
10:	learn: 5.3858839	total: 472ms	remaining: 42.5s
11:	learn: 5.3799524	total: 521ms	remaining: 42.9s
12:	learn: 5.3741957	total: 569ms	remaining: 43.2s
13:	learn: 5.3685716	total: 614ms	remaining: 43.2s
14:	learn: 5.3630611	total: 658ms	remaining: 43.2s
15:	learn: 5.3576345	total: 708ms	remaining: 43.5s
16:	learn: 5.3523038	total: 749ms	remaining: 43.3s
17:	learn: 5.3470987	total: 787ms	remaining: 42.9s
18:	learn: 5.3420818	total: 818ms	remaining: 42.3s
19:	learn: 5.3368226	total: 845ms	remai

In [41]:
predicteds = classifier.predict(X_test)

In [42]:
predicteds

array([7.56655895, 6.4237394 , 4.90627025, ..., 5.42466878, 4.46584159,
       1.48448367])

In [43]:
mean_absolute_error(Y_test,predicteds)

2.856107537099404